# Presents all data from the Frequency Response Magnetophosphene study
## effect of ELF-MF (up to 300Hz) on magnetophosphene perception threshold

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

rcParams['figure.figsize'] = 10, 10
sns.set_style("whitegrid")

%matplotlib inline

In [ ]:
filename = '../DATA/All_Thresholds.txt'
data = pd.read_csv(filename,sep='\t')
data['Perceived'] = data.Threshold > 0  # perceived if threshold value
data.head()

## extract percentage of perception for each frequency

In [ ]:
frequencies = data.Frequency.unique()
frequencies.sort()

In [ ]:
distr_perceived = pd.Series(index = frequencies)

for fr in frequencies:
    df = data.loc[data['Frequency']==fr,'Perceived']
    if df.any():
        percentage = df.value_counts()[True]/df.value_counts().sum()
    else:
        percentage = 0
    
    distr_perceived[fr] = percentage

In [ ]:
df

In [ ]:
fig, ax  = plt.subplots()
ax.plot(distr_perceived.index, distr_perceived, color='blue', lw='2')
ax.fill_between(distr_perceived.index, 0, distr_perceived, alpha=.3)

ax.plot(distr_perceived[distr_perceived > 0.8], 'k.', ms=15)
# ax.plot(distr_perceived[distr_perceived > 0.95], 'r.')

plt.xticks(frequencies)
plt.xlabel('MF Frequency (Hz)')
plt.ylabel('Perception rate %')

In [ ]:
# plt.scatter(data.Frequency, data.Threshold)
plt.figure()
sns.stripplot(x="Frequency", y="Threshold", data=data, color='black')

In [ ]:
plt.figure()
plt.plot(data.index, data.Threshold > 0, 'r.')
plt.plot(data.index, data['Yes/No'], 'bo', fillstyle='none')

## polynomial regression

In [ ]:
data = data.set_index('Frequency')
idxFreq = distr_perceived[distr_perceived >0.8].index
data

In [ ]:
# new_data = data.loc[idxFreq].dropna(subset=["Threshold"]) ## remove rows with no perception
new_data = data[data['Perceived']].loc[idxFreq]

### Using sklearn linear_model - polynomialFeatures
(can be done with numpy poly1d)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
Freq = new_data.index.to_numpy()
Thres = new_data.Threshold
nFreq = np.arange(0,150)

x = Freq[:, np.newaxis]
y = Thres[:, np.newaxis]
new_x = np.arange(0,150)[:,np.newaxis]

polynomial_features = PolynomialFeatures(degree=2)
x_poly = polynomial_features.fit_transform(x)
newx_poly = polynomial_features.fit_transform(new_x)

model = LinearRegression()

# linear regression
model.fit(x, y)
y_pred = model.predict(new_x)

# polynomial regression
model.fit(x_poly, y)
ypol_predict = model.predict(newx_poly)

In [ ]:
plt.figure()
plt.scatter(data.index, data.Threshold, color='k')

plt.plot(Freq, Thres, 'r.', ms=5)
plt.plot(new_x, y_pred, 'blue')
plt.plot(new_x, ypol_predict, 'm')

### Using StatsModels for statistics
https://ostwalprasad.github.io/machine-learning/Polynomial-Regression-using-statsmodel.html

In [ ]:
import statsmodels.api as sm

In [ ]:
s_model = sm.OLS(y, x_poly).fit()
sy_pred = s_model.predict(x_poly)

In [ ]:
s_model.summary()

In [ ]:
predictions = s_model.get_prediction(newx_poly)
toto = predictions.summary_frame(alpha=0.01)

In [ ]:
toto

In [ ]:
predictions = s_model.get_prediction(newx_poly)

In [ ]:
conf_95 = predictions.summary_frame()
conf_01 = predictions.summary_frame(alpha=0.01)

fig, ax1 = plt.subplots()
plt.plot(new_x,conf_95['mean'],'b', linewidth=3, label="Mean")
plt.plot(new_x,conf_95['mean_ci_lower'],'cyan')
plt.plot(new_x,conf_95['mean_ci_upper'],'cyan')
plt.fill_between(conf_95.index, conf_95.mean_ci_lower, conf_95.mean_ci_upper, alpha=.3)

plt.plot(new_x,conf_01.obs_ci_upper,':',label="99%")
plt.plot(new_x,conf_95.obs_ci_upper,':',label="87.5%")
plt.plot(new_x,conf_95.obs_ci_lower,':',label="2.5%")
plt.plot(new_x,conf_01.obs_ci_lower,':',label="1%")

plt.plot(data[data['Perceived']].index, data[data['Perceived']].Threshold, 'k.')
plt.plot(Freq, Thres, 'r.', ms=2)

plt.xlabel('MF Frequency (Hz)')
plt.ylabel('Threshold (mT)')

plt.legend()

# plot perception rate
if False:
    ax2 = ax1.twinx()
    ax2.plot(distr_perceived.index, distr_perceived, color='blue', lw='0.5')
    ax2.fill_between(distr_perceived.index, 0, distr_perceived, alpha=.1)

In [ ]:
fig.savefig('../ThresholdModels.png',dpi=300)

In [ ]:
# data[data.Threshold > 70].to_csv('HighThreshold.csv')

___
## __Explore first 12 subjects__

In [ ]:
data.ID.unique()

In [ ]:
bID = ['P1', 'P2', 'P3', 'P4', 'P5', 'P7', 'P8', 'P9', 'P10', 'P12']

In [ ]:
df = data.loc[data.ID.isin(bID),:]

In [ ]:
plt.plot(data.index, data.Threshold, 'k.')
plt.plot(df.index, df.Threshold, 'y.', ms=10)
plt.xticks(ticks=frequencies, labels=frequencies)
plt.hlines(y=6, xmin=0, xmax=300)

___
## __Explore P50 with low thresholds__

In [ ]:
data[(data.Threshold < 6) & (data.Threshold > 0)].ID.unique()

In [ ]:
plt.plot(data[data.ID=='P50'].index, data[data.ID=='P50'].Threshold, 'b.', ms=20)

---
# __Threshold Methods P59__

In [ ]:
pd.options.display.max_rows = 50

In [ ]:
data[data.ID=='P59'].sort_index()

## read different threshold assessment 

In [ ]:
df59_b = pd.read_csv("B_Threshold.csv",header=None, names='B')
df59_mf = pd.read_csv("MF_Threshold.csv",header=None, names=["MF"])
df59_rms = pd.read_csv("rms_Threshold.csv",header=None, names=["rms"])
new_df = data[['ID', 'Threshold', 'Yes/No']].query('ID == "P59"').reset_index()
new_df.insert(3, 'new_th',new_df['Yes/No'].mul(df59_b.B))

In [ ]:
new_df

In [ ]:
plt.plot(new_df['new_th'],'ro', label='model_peak')
plt.plot(df59_rms.rms.mul(new_df['Yes/No']),'r*', label='model_rms')
plt.plot(new_df['Threshold'],'k.', label='dial coefficient')
plt.plot(df59_mf.MF.mul(new_df['Yes/No']),'b.', label='MF 16cm')


plt.legend()
plt.xlabel('iterations')
plt.ylabel('Threshold rms  - B (mT)')